# Introduction/buisness problem

### A lot of potential thoughts and considerations are given to the locations of housing and real estate investments. We need to look at the value of a place which is eventually determined by proximity to important/busy areas in the city or place of interest. In that case what if we could assort neighbourhoods on the basis of the popular locations they have and that could help investors know the right place to bank thier assets

# Explaination 

### Toronto is a fast growing finacial hub. With increasing city centres and spurling residential hubs. This makes real estate one of many growing business ventures toronto has had. The city is well connected and well planned and is financially flourishing and that aspect brings in a lot of investors. Hence determining potential investment intrests is definitely benificial 

## Who are our stakeholders 
### Anyone who's intrested in real estate market in the reigon, it can be individuals or property brokers or even the residents who would want to know if they are in the right neibourhood acoording to thier intrests and needs 

## What am i trying to do 
### I will use datasets regarding neighbourhoods in toronto and comlement them with foursquare API data on popular places or hotspots close to the places of interest then i can cluster the neighbourhoods with thier anemities/ hotspots and map them using folium 

.

.

# Data (exploration)

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

### getting data 

In [3]:
from bs4 import BeautifulSoup as bs
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text # get the html text by requests
soup = bs(url,'lxml') 

In [4]:
table =soup.find_all('table', class_="wikitable sortable")
soup_cont = table[0].find_all('tr')  
print(soup_cont[1])  
clean = soup_cont[1].find_all('td') 
Postalcode = clean[0].text.strip("\n")   
Borough = clean[1].text.strip("\n")
Neighborhood = clean[2].text.strip("\n")

print('This is the first row of data :',[Postalcode,Borough,Neighborhood])

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
This is the first row of data : ['M1A', 'Not assigned', 'Not assigned']


In [7]:
# insert the data labels of table
labels = {'PostalCode':[],'Borough':[],'Neighborhood':[]} 

# loop to get all of the data
for i in range(len(soup_cont)-1):  
    labels['PostalCode'].append(soup_cont[i+1].find_all('td')[0].text.strip("\n"))
    labels['Borough'].append(soup_cont[i+1].find_all('td')[1].text.strip("\n"))
    labels['Neighborhood'].append(soup_cont[i+1].find_all('td')[2].text.strip("\n"))

# dict to df
table = pd.DataFrame(data=labels) 
cols = ['PostalCode','Borough','Neighborhood']
table = table.loc[:,cols]

table.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
table

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [17]:
data = table[table.Borough != 'Not assigned']
data.loc[df.Neighborhood == 'Not assigned']
data[data.Neighborhood == 'Not assigned'].index[0] #　Find target index
data.loc[data[data.Neighborhood == 'Not assigned'].index[0],'Neighborhood'] = 'Queens Park'
data.head(5)

# that's a warning i couldn't negate, but it doesn't affect any process

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [18]:
data

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queens Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [20]:
## group by neighbourhoods
data = df
data = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "[%s]" % ', '.join(x))  
data = data.reset_index()  # 
data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


,

In [21]:
dset=pd.read_csv('http://cocl.us/Geospatial_data')

In [23]:
data['Latitude']= dset['Latitude'] # adding the lattitude and longitude columns to our initial dataset
data['Longitude']= dset['Longitude']
toronto = data # named it toronto because its the city location data
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [24]:
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize

!pip install folium
import folium 
print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Folium installed
Libraries imported.


In [25]:
CLIENT_ID = 'DU5GASMW4BUEWN4S1K2EUX1HZQQA21QLCF5DUJ0QSDKPBYML' # your Foursquare ID
CLIENT_SECRET = 'QSAAGXADTJUDUWDYZ3AAW20JQO3YHKBEWQ3CPY0GZONMKPNE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DU5GASMW4BUEWN4S1K2EUX1HZQQA21QLCF5DUJ0QSDKPBYML
CLIENT_SECRET:QSAAGXADTJUDUWDYZ3AAW20JQO3YHKBEWQ3CPY0GZONMKPNE


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

[Rouge, Malvern]
[Highland Creek, Rouge Hill, Port Union]
[Guildwood, Morningside, West Hill]
[Woburn]
[Cedarbrae]
[Scarborough Village]
[East Birchmount Park, Ionview, Kennedy Park]
[Clairlea, Golden Mile, Oakridge]
[Cliffcrest, Cliffside, Scarborough Village West]
[Birch Cliff, Cliffside West]
[Dorset Park, Scarborough Town Centre, Wexford Heights]
[Maryvale, Wexford]
[Agincourt]
[Clarks Corners, Sullivan, Tam O'Shanter]
[Agincourt North, L'Amoreaux East, Milliken, Steeles East]
[L'Amoreaux West]
[Upper Rouge]
[Hillcrest Village]
[Fairview, Henry Farm, Oriole]
[Bayview Village]
[Silver Hills, York Mills]
[Newtonbrook, Willowdale]
[Willowdale South]
[York Mills West]
[Willowdale West]
[Parkwoods]
[Don Mills North]
[Flemingdon Park, Don Mills South]
[Bathurst Manor, Downsview North, Wilson Heights]
[Northwood Park, York University]
[CFB Toronto, Downsview East]
[Downsview West]
[Downsview Central]
[Downsview Northwest]
[Victoria Village]
[Woodbine Gardens, Parkview Hill]
[Woodbine Heig

In [31]:
print(toronto_venues.shape)
toronto_venues

(1332, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"[Rouge, Malvern]",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"[Rouge, Malvern]",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
5,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
7,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
8,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,chatr Mobile,43.765917,-79.191672,Tech Startup
9,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


### this is the data we needed we have all the famous venues corresponding to neighbourhoods in the city 

In [39]:
latitude = toronto.Latitude.mean()
longitude = toronto.Longitude.mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_venues['Neighborhood Latitude'], toronto_venues['Neighborhood Longitude'], toronto_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto